In [1]:
from google.colab import files
uploaded = files.upload()
from ipyleaflet import Map, Marker, Polygon, FullScreenControl, LegendControl
from ipywidgets import Layout
import pandas as pd
from scipy.spatial import ConvexHull

PATH_TO_SAVE = "convex_hulls.csv"
points_df = pd.read_csv("/content/points.csv")

class MapRenderer:
    def __init__(self, district_data: pd.DataFrame, points_data: pd.DataFrame):
        self.__points_data = points_data
        self.__district_data = district_data

    def get_map(self) -> Map:
        center_lat = self.__points_data['lat'].median()
        center_lon = self.__points_data['lon'].median()

        m = Map(center=[center_lat, center_lon], zoom=12, layout=Layout(width='100%', height='900px'))


        for _, row in self.__district_data.iterrows():
            district_name = row['district']
            center_lat, center_lon = row['center']
            marker = Marker(location=[center_lat, center_lon], title=district_name)
            m.add_layer(marker)
            points = eval(row['points'])
            polygon = Polygon(locations=points, color=row['color'], fill_opacity=0.2, fill_color=row['color'])
            m.add_layer(polygon)

        names = {row['district']: row['color'] for _, row in self.__district_data.iterrows()}
        names_control = LegendControl(legend=names, name='Районы Казани')
        m.add_control(names_control)

        return m

class ConvexHullBuilder:
    def __init__(self, points: pd.DataFrame):
        self.__points = points

    def get_convex_hull(self) -> pd.DataFrame:
        convex_hulls = []
        grouped_data = self.__points.groupby('district')

        district_colors = {
            'Советский р-н': 'green',
            'Кировский р-н': 'blue',
            'Приволжский р-н': 'yellow',
            'Московский р-н': 'red',
            'Авиастроительный р-н': 'pink',
            'Ново-Савиновский р-н': 'white',
            'Вахитовский р-н': 'black',
            'Верхнеуслонский р-н': 'brown',
        }

        for district, group in grouped_data:
            points = group[['lat', 'lon']].values
            hull = ConvexHull(points)
            hull_points = points[hull.vertices]
            center = tuple(hull_points.mean(axis=0))
            color = district_colors.get(district)

            convex_hulls.append({
                'district': district,
                'points': hull_points.tolist(),
                'center': center,
                'color': color
            })


        return pd.DataFrame(convex_hulls)

builder = ConvexHullBuilder(points_df)

result_df = builder.get_convex_hull()

result_df.to_csv(PATH_TO_SAVE, index=False)


Saving points.csv to points.csv


In [2]:
points_df = pd.read_csv("points.csv")
districts_df = pd.read_csv("convex_hulls.csv")

districts_df['center'] = districts_df['center'].apply(eval)

renderer = MapRenderer(districts_df, points_df)
renderer.get_map()

Map(center=[55.79462976, 49.15689314], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…